In [67]:
## Dependencies

import pandas as pd
import os


In [68]:
## laod test data

df = pd.read_csv('TEMP\clean_tables\game_level\_2010_games.csv')

In [69]:
## NEWWLY REFACTORED FUNCTION MON 1PM CHATGPT

import pandas as pd

def process_stats(df):
    ## Create a column with the result of each game win, loss, or tie
    def calc_result(row):
        if row['teamScore'] > row['opponentScore']:
            return 'W'
        elif row['teamScore'] < row['opponentScore']:
            return 'L'
        else:
            return 'T'
    df['result'] = df.apply(calc_result, axis=1)

    def process_slice(df_slice, prefix):
        grouped = df_slice.groupby('teamName')

        # Save value counts to a new dataframe with the team name as the index and columns for each result type
        results_frame = pd.DataFrame(grouped['result'].value_counts()).rename(columns={'result':'count'}).reset_index()

        # Reshape results into separate columns for each result type using pivot table
        results_frame = results_frame.pivot(index='teamName', columns='result', values='count')

        # Replace NaN values with 0
        results_frame = results_frame.fillna(0)

        # Rename the columns
        results_frame.columns = [f'{prefix}_{col}' for col in results_frame.columns]

        # Rest of the function
        ...



        stat = grouped.agg({
            'teamScore': ['count', 'mean', 'median', 'sum'],
            'opponentScore': ['count', 'mean', 'median', 'sum']
        }).reset_index()

        # simplify the column names
        stat.columns = ['teamName', 
                        
                        f'{prefix}_games_played',
                        f'{prefix}_runs_scored_mean',
                        f'{prefix}_runs_scored_median',
                        f'{prefix}_runs_scored_total',

                        f'{prefix}_runs_allowed_count',
                        f'{prefix}_runs_allowed_mean',
                        f'{prefix}_runs_allowed_median',
                        f'{prefix}_runs_allowed_total']

        # Add the result count columns to the stat dataframe
        stat = stat.merge(results_frame, how='left', on='teamName')

        # Calculate winning percentage and run differential (total and mean)
        stat[f'{prefix}_win_pct'] = stat[f'{prefix}_W'] / stat[f'{prefix}_games_played']
        stat[f'{prefix}_run_diff_total'] = stat[f'{prefix}_runs_scored_total'] - stat[f'{prefix}_runs_allowed_total']
        stat[f'{prefix}_run_diff_mean'] = stat[f'{prefix}_runs_scored_mean'] - stat[f'{prefix}_runs_allowed_mean']

        return stat


    # Process data slices
    slices = {
        'playoff': df[df['postSeasonInfo'].notnull()],
        'regular_season': df[df['postSeasonInfo'].isnull()],
        'home': df[(df['postSeasonInfo'].isnull()) & (df['homeOrAway'] == 'H')],
        'away': df[(df['postSeasonInfo'].isnull()) & (df['homeOrAway'] == 'A')],
    }
    combined_stat = None
    for prefix, df_slice in slices.items():
        slice_stat = process_slice(df_slice, prefix)
        if combined_stat is None:
            combined_stat = slice_stat
        else:
            combined_stat = combined_stat.merge(slice_stat, how='left', on='teamName')

    return combined_stat


In [70]:
## Run the function on the test data and display the results

test_df = process_stats(df)

## Display the results
test_df.head()






,teamName,playoff_games_played,playoff_runs_scored_mean,playoff_runs_scored_median,playoff_runs_scored_total,playoff_runs_allowed_count,playoff_runs_allowed_mean,playoff_runs_allowed_median,playoff_runs_allowed_total,playoff_L,...,away_runs_allowed_count,away_runs_allowed_mean,away_runs_allowed_median,away_runs_allowed_total,away_L,away_T,away_W,away_win_pct,away_run_diff_total,away_run_diff_mean
0,Grand Rapids Covenant Christian,1,4.0,4.0,4.0,1,5.0,5.0,5.0,1.0,...,5,2.400000,1.0,12.0,2.0,1.0,3.0,0.600000,23.0,4.600000
1,Grandville Calvin Christian,1,9.0,9.0,9.0,1,5.0,5.0,5.0,0.0,...,1,4.000000,4.0,4.0,0.0,1.0,1.0,1.000000,5.0,5.000000
2,Wyoming Lee,2,5.0,5.0,10.0,2,6.5,6.5,13.0,1.0,...,13,5.923077,5.0,77.0,3.0,0.0,10.0,0.769231,44.0,3.384615


In [71]:
### SAVE A TEMP DATAFRAME TO CSV

test_df.to_csv('TEMP\clean_tables\game_level\_2010_games_stats_Y_S_L_book.csv', index=False)

In [72]:
## Block of all the stat processing code to make into a function
def process_stats(df):
    ## Create a column WIth the result of each game win, loss or tie

    def calc_result(row):
        if row['teamScore'] > row['opponentScore']:
            return 'W'
        elif row['teamScore'] < row['opponentScore']:
            return 'L'
        else:
            return 'T'

    df['result'] = df.apply(calc_result, axis=1)

    ## New Stragety - split the dataframe into playoff games, regular season games

    ## playoff dataframe
    playoff_df = df[df['postSeasonInfo'].notnull()]


    playoff_df['homeOrAway'] = 'N'


    ## regular season dataframe
    regular_season_df = df[df['postSeasonInfo'].isnull()]

    # Create home and away dataframes for the regular season dataframes
    home_df = regular_season_df[regular_season_df['homeOrAway'] == 'H']   # home games
    away_df = regular_season_df[regular_season_df['homeOrAway'] == 'A']   # away games


    # Create a new dataframe with the summary stats for each team and each dataframe
    ####### PLAYOFFS #######
    grouped = playoff_df.groupby('teamName')

    ## Save value counts to a new dataframe with the team name as the index and columns for each result type
    results_frame = pd.DataFrame(grouped['result'].value_counts())

    # transpose results into seperate columns for each result type
    results_frame = results_frame.unstack(level=-1)

    playoff_stat = grouped.agg({
        'teamScore': ['count', 'mean', 'median', 'sum'],
        'opponentScore': ['count', 'mean', 'median', 'sum']
    }).reset_index()




    # playoff_stat = df[['teamName', 'teamId', 'leagueName']].drop_duplicates()

    # simplify the column names
    playoff_stat.columns = ['teamName', 
                            
                            'playoff_games_played',
                            'playoff_runs_scored_mean',
                            'playoff_runs_scored_median',
                            'playoff_runs_scored_total',

                            'playoff_runs_allowed_count',
                            'playoff_runs_allowed_mean',
                            'playoff_runs_allowed_median',
                            'playoff_runs_allowed_total']


    ## Add the result count columns to the playoff_stat dataframe
    playoff_stat = playoff_stat.merge(results_frame, how='left', on='teamName')

    ## Rename the columns
    playoff_stat.columns = ['teamName',
                            
                            'playoff_games_played',
                            'playoff_runs_scored_mean',
                            'playoff_runs_scored_median',
                            'playoff_runs_scored_total',

                            'playoff_runs_allowed_count',
                            'playoff_runs_allowed_mean',
                            'playoff_runs_allowed_median',
                            'playoff_runs_allowed_total',
                            
                            'playoff_losses',
                            'playoff_wins'
                            ]

    # Calculate winning percentage and run differential (total and mean)
    playoff_stat['playoff_win_pct'] = playoff_stat['playoff_wins'] / playoff_stat['playoff_games_played']
    playoff_stat['playoff_run_diff_total'] = playoff_stat['playoff_runs_scored_total'] - playoff_stat['playoff_runs_allowed_total']
    playoff_stat['playoff_run_diff_mean'] = playoff_stat['playoff_runs_scored_mean'] - playoff_stat['playoff_runs_allowed_mean']

    ## Reorder the columns so the result columns right after the games played column
    playoff_stat = playoff_stat[['teamName',
                            'playoff_games_played',
                            'playoff_wins',
                            'playoff_losses',
                            'playoff_win_pct',
                            'playoff_runs_scored_mean', 
                            'playoff_runs_scored_median',
                            'playoff_runs_scored_total',
                            'playoff_runs_allowed_mean',
                            'playoff_runs_allowed_median',
                            'playoff_runs_allowed_total',
                            'playoff_run_diff_total',
                            'playoff_run_diff_mean'
                            ]]


    ### Regular Season Stats#####

    ## Create a new dataframe with the summary stats for each team and each dataframe
    grouped = regular_season_df.groupby('teamName')

    # Save the total games played for each team
    regular_season_games_played = grouped['teamName'].count()

    ## Save value counts to a new dataframe with the team name as the index and columns for each result type
    results_frame = pd.DataFrame(grouped['result'].value_counts())

    # # transpose results into seperate columns for each result type W L T
    results_frame = results_frame.unstack(level=-1)

    # add the regular_season_games_played
    results_frame['regular_season_games_played'] = regular_season_games_played

    # Rename reults columns
    results_frame.columns = ['regular_season_losses',
                            'regular_season_ties',
                            'regular_season_wins',
                            'regular_season_games_played']


    ## Save the summary stats to a new dataframe
    regular_season_stat = grouped.agg({
        'teamScore': ['mean', 'median', 'sum'],
        'opponentScore': ['mean', 'median', 'sum']
    }).reset_index()



    # simplify the column names
    regular_season_stat.columns = ['teamName',
                            
                            'regular_season_runs_scored_mean',
                            'regular_season_runs_scored_median',
                            'regular_season_runs_scored_total',
                            
                            'regular_season_runs_allowed_mean',
                            'regular_season_runs_allowed_median',
                            'regular_season_runs_allowed_total']

    ## Add the result count columns to the regular_season_stat dataframe
    regular_season_stat = regular_season_stat.merge(results_frame, how='left', on='teamName')


    # Calculate winning percentage and run differential (total and mean)
    regular_season_stat['regular_season_win_pct'] = regular_season_stat['regular_season_wins'] / regular_season_stat['regular_season_games_played']
    regular_season_stat['regular_season_run_diff_total'] = regular_season_stat['regular_season_runs_scored_total'] - regular_season_stat['regular_season_runs_allowed_total']
    regular_season_stat['regular_season_run_diff_mean'] = regular_season_stat['regular_season_runs_scored_mean'] - regular_season_stat['regular_season_runs_allowed_mean']

    ## Reorder the columns so the result columns right after the games played column
    regular_season_stat = regular_season_stat[['teamName',
                            'regular_season_games_played',
                            'regular_season_wins',
                            'regular_season_losses',
                            'regular_season_ties',
                            'regular_season_win_pct',
                            'regular_season_runs_scored_mean',
                            'regular_season_runs_scored_median',
                            'regular_season_runs_scored_total',
                            'regular_season_runs_allowed_mean',
                            'regular_season_runs_allowed_median',
                            'regular_season_runs_allowed_total',
                            'regular_season_run_diff_total',
                            'regular_season_run_diff_mean'
                            ]]


    ### Combine the playoff and regular season stats into one dataframe

    ## Merge the playoff and regular season stats into one dataframe
    combined_stat = playoff_stat.merge(regular_season_stat, how='left', on='teamName')

    combined_stat.head()

    ## Create a new dataframe with the summary stats for home and away games
    # home dataframe

    grouped = home_df.groupby('teamName')

    # head
    grouped.head()

    ## Save the summary stats to a new dataframe
    home_stat = grouped.agg({
        'teamScore': ['mean', 'median', 'sum'],
        'opponentScore': ['mean', 'median', 'sum']
    }).reset_index()

    # simplify the column names
    home_stat.columns = ['teamName',
                        'home_runs_scored_mean',
                        'home_runs_scored_median',
                        'home_runs_scored_total',

                        'home_runs_allowed_mean',
                        'home_runs_allowed_median',
                        'home_runs_allowed_total']

    ## Add the result count columns to the home_stat dataframe
    home_stat = home_stat.merge(results_frame, how='left', on='teamName')


    # Save the total games played for each team
    home_games_played = grouped['teamName'].count()

    ## Save value counts to a new dataframe with the team name as the index and columns for each result type
    results_frame = pd.DataFrame(grouped['result'].value_counts())

    # # # transpose results into seperate columns for each result type W L T
    results_frame = results_frame.unstack(level=-1)

    # simplify the column names
    results_frame.columns = ['home_losses',
                            'home_ties',
                            'home_wins']

    # # add the home_games_played
    results_frame['home_games_played'] = home_games_played

    # # Reorder reults columns
    results_frame = results_frame[['home_games_played',
                                'home_wins',
                                'home_losses',
                                'home_ties']]





    # Calculate winning percentage
    results_frame['home_win_pct'] = results_frame['home_wins'] / results_frame['home_games_played']

    ## add the results columns to the home_stat dataframe
    home_stat = home_stat.merge(results_frame, how='left', on='teamName')

    # Calculate run differential (total and mean)
    home_stat['home_run_diff_total'] = home_stat['home_runs_scored_total'] - home_stat['home_runs_allowed_total']
    home_stat['home_run_diff_mean'] = home_stat['home_runs_scored_mean'] - home_stat['home_runs_allowed_mean']



    # drop the rehular season games played column
    # home_stat.drop(columns=['regular_season_losses','regular_season_ties','regular_season_wins','regular_season_games_played'], inplace=True)


    # reorder the columns
    home_stat = home_stat[['teamName',
                            'home_games_played',
                            'home_wins',
                            'home_losses',
                            'home_ties',
                            'home_win_pct',

                            'home_runs_scored_mean',
                            'home_runs_scored_median',
                            'home_runs_scored_total',

                            'home_runs_allowed_mean',
                            'home_runs_allowed_median',
                            'home_runs_allowed_total',

                            'home_run_diff_total',
                            'home_run_diff_mean'
                            ]]


    ### Merge the home stats into the combined dataframe
    combined_stat = combined_stat.merge(home_stat, how='left', on='teamName')


    ### Calculate the away stats

    ## Create a new dataframe with the summary stats for away games
    # away dataframe

    grouped = away_df.groupby('teamName')

    # head
    grouped.head()


    ## Save the summary stats to a new dataframe
    away_stat = grouped.agg({
        'teamScore': ['mean', 'median', 'sum'],
        'opponentScore': ['mean', 'median', 'sum']
    }).reset_index()

    # simplify the column names
    away_stat.columns = ['teamName',
                        'away_runs_scored_mean',
                        'away_runs_scored_median',
                        'away_runs_scored_total',

                        'away_runs_allowed_mean',
                        'away_runs_allowed_median',
                        'away_runs_allowed_total']

    ## Add the result count columns to the home_stat dataframe
    away_stat = away_stat.merge(results_frame, how='left', on='teamName')


    # Save the total games played for each team
    away_games_played = grouped['teamName'].count()

    ## Save value counts to a new dataframe with the team name as the index and columns for each result type
    results_frame = pd.DataFrame(grouped['result'].value_counts())

    # # # transpose results into seperate columns for each result type W L T
    results_frame = results_frame.unstack(level=-1)

    # simplify the column names
    results_frame.columns = ['away_losses',
                            'away_ties',
                            'away_wins']

    # # add the home_games_played
    results_frame['away_games_played'] = away_games_played

    # # Reorder reults columns
    results_frame = results_frame[['away_games_played',
                                'away_wins',
                                'away_losses',
                                'away_ties']]





    # Calculate winning percentage
    results_frame['away_win_pct'] = results_frame['away_wins'] / results_frame['away_games_played']

    ## add the results columns to the home_stat dataframe
    away_stat = away_stat.merge(results_frame, how='left', on='teamName')

    # Calculate run differential (total and mean)
    away_stat['away_run_diff_total'] = away_stat['away_runs_scored_total'] - away_stat['away_runs_allowed_total']
    away_stat['away_run_diff_mean'] = away_stat['away_runs_scored_mean'] - away_stat['away_runs_allowed_mean']



    # drop the rehular season games played column
    # away_stat.drop(columns=['regular_season_losses','regular_season_ties','regular_season_wins','regular_season_games_played'], inplace=True)


    # reorder the columns
    away_stat = away_stat[['teamName',
                            'away_games_played',
                            'away_wins',
                            'away_losses',
                            'away_ties',
                            'away_win_pct',

                            'away_runs_scored_mean',
                            'away_runs_scored_median',
                            'away_runs_scored_total',

                            'away_runs_allowed_mean',
                            'away_runs_allowed_median',
                            'away_runs_allowed_total',

                            'away_run_diff_total',
                            'away_run_diff_mean'
                            ]]


    ## Merge the away stats into the combined dataframe
    combined_stat = combined_stat.merge(away_stat, how='left', on='teamName')

    ## End the function
    return combined_stat








In [73]:
### Run function to create the combined stat dataframe
test_df = process_stats(df)

test_df.head()



C:\Users\Justin\AppData\Local\Temp\ipykernel_14160\49686019.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playoff_df['homeOrAway'] = 'N'
C:\Users\Justin\AppData\Local\Temp\ipykernel_14160\49686019.py:67: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  playoff_stat = playoff_stat.merge(results_frame, how='left', on='teamName')


,teamName,playoff_games_played,playoff_wins,playoff_losses,playoff_win_pct,playoff_runs_scored_mean,playoff_runs_scored_median,playoff_runs_scored_total,playoff_runs_allowed_mean,playoff_runs_allowed_median,...,away_ties,away_win_pct,away_runs_scored_mean,away_runs_scored_median,away_runs_scored_total,away_runs_allowed_mean,away_runs_allowed_median,away_runs_allowed_total,away_run_diff_total,away_run_diff_mean
0,Grand Rapids Covenant Christian,1,NaN,1.0,NaN,4.0,4.0,4.0,5.0,5.0,...,1.0,0.500000,7.000000,7.0,35.0,2.400000,1.0,12.0,23.0,4.600000
1,Grandville Calvin Christian,1,1.0,NaN,1.0,9.0,9.0,9.0,5.0,5.0,...,1.0,0.500000,9.000000,9.0,9.0,4.000000,4.0,4.0,5.0,5.000000
2,Wyoming Lee,2,1.0,1.0,0.5,5.0,5.0,10.0,6.5,6.5,...,NaN,0.769231,9.307692,8.0,121.0,5.923077,5.0,77.0,44.0,3.384615


In [74]:
test_df.describe()

,playoff_games_played,playoff_wins,playoff_losses,playoff_win_pct,playoff_runs_scored_mean,playoff_runs_scored_median,playoff_runs_scored_total,playoff_runs_allowed_mean,playoff_runs_allowed_median,playoff_runs_allowed_total,...,away_ties,away_win_pct,away_runs_scored_mean,away_runs_scored_median,away_runs_scored_total,away_runs_allowed_mean,away_runs_allowed_median,away_runs_allowed_total,away_run_diff_total,away_run_diff_mean
count,3.000000,2.0,2.0,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,2.0,3.000000,3.000000,3.0,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,1.333333,1.0,1.0,0.750000,6.000000,6.000000,7.666667,5.500000,5.500000,7.666667,...,1.0,0.589744,8.435897,8.0,55.000000,4.107692,3.333333,31.000000,24.000000,4.328205
std,0.577350,0.0,0.0,0.353553,2.645751,2.645751,3.214550,0.866025,0.866025,4.618802,...,0.0,0.155440,1.253004,1.0,58.617404,1.764006,2.081666,40.037482,19.519221,0.841291
min,1.000000,1.0,1.0,0.500000,4.000000,4.000000,4.000000,5.000000,5.000000,5.000000,...,1.0,0.500000,7.000000,7.0,9.000000,2.400000,1.000000,4.000000,5.000000,3.384615
25%,1.000000,1.0,1.0,0.625000,4.500000,4.500000,6.500000,5.000000,5.000000,5.000000,...,1.0,0.500000,8.000000,7.5,22.000000,3.200000,2.500000,8.000000,14.000000,3.992308
50%,1.000000,1.0,1.0,0.750000,5.000000,5.000000,9.000000,5.000000,5.000000,5.000000,...,1.0,0.500000,9.000000,8.0,35.000000,4.000000,4.000000,12.000000,23.000000,4.600000
75%,1.500000,1.0,1.0,0.875000,7.000000,7.000000,9.500000,5.750000,5.750000,9.000000,...,1.0,0.634615,9.153846,8.5,78.000000,4.961538,4.500000,44.500000,33.500000,4.800000
max,2.000000,1.0,1.0,1.000000,9.000000,9.000000,10.000000,6.500000,6.500000,13.000000,...,1.0,0.769231,9.307692,9.0,121.000000,5.923077,5.000000,77.000000,44.000000,5.000000


In [75]:
## Load each csv indivitually and pass it to the function

# Load csv

df = pd.read_csv('TEMP\clean_tables\game_level\_2014_games.csv')

In [76]:
## Run the df through the function and save it to a new dataframe and eventually a csv
result_df = process_stats(df)

C:\Users\Justin\AppData\Local\Temp\ipykernel_14160\49686019.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playoff_df['homeOrAway'] = 'N'
C:\Users\Justin\AppData\Local\Temp\ipykernel_14160\49686019.py:67: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  playoff_stat = playoff_stat.merge(results_frame, how='left', on='teamName')


ValueError: Length mismatch: Expected axis has 12 elements, new values have 11 elements

In [ ]:
## Check output
# result_df.info()

## Save the dataframe to a csv
result_df.to_csv('TEMP\clean_tables\game_level\processed\_2014_team_stats.csv', index=False)

In [ ]:
## LOOP NOT WORKING

# ## create a list of files to read as dataframes and pass to the function

# file_dir = 'TEMP\clean_tables\game_level'
# file_list = os.listdir(file_dir)

# file_list = [file for file in file_list if file.endswith('.csv')]

# ## Load the files as a list of dataframes

# df_list = []

# for file in file_list:
#     df = pd.read_csv(file_dir + '\\' + file)
#     df_list.append(df)

# ## Show the list of dataframes
# # df_list




In [ ]:
# ## Print the shape of all the dataframes with the file name
# for df in df_list:
#     # add the file name to the dataframe
#     df['file_name'] = file

    
#     print(df['file_name'][0], df.shape, df.columns[0])



In [ ]:
# ## Pass the list of dataframes to the function and save the results of each to a csv file with the name as the original file + 'summary_stats.csv'

# for df in df_list:

#     # Parse the file using the function
#     df = process_stats(df)

#     # save the resulting dataframe to a csv file
#     df.to_csv(file_dir + '\\' + 'processed\\' + file + '_summary_stats.csv', index=False)

#     # print the file name to show progress
#     print(file + ' saved')

    